In [42]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import warnings
import gc
import copy
from itertools import product
from IPython.display import display
from IPython.display import set_matplotlib_formats
import datetime
import tqdm
from scipy import stats
from sklearn.preprocessing import LabelEncoder
set_matplotlib_formats('svg')

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', 100)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = [u'SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [43]:
# ['销售订单编号', '物料编号', '是否三包描述', '基本计量单位', '产品组名称', '产品组代码', 
# '创建日期', 'year', 'month', 'week', 'day', 'weekday', '订单数量', '订单金额', '物料描述', 
# '是否进口件', '物料类别', '周期（天）下计划后的交货周期', '在产情况', '吨位区间', '设备类型']
df = pd.read_csv('gongqi.csv', encoding='utf-8-sig',
                 usecols=['物料编号', '基本计量单位', 'year', 'month', 'week', 'day', '创建日期', '订单金额', '物料描述', '是否三包描述',
                          '订单数量', '是否进口件', '物料类别', '周期（天）下计划后的交货周期', '在产情况', '吨位区间', '设备类型'])
print(len(df['物料编号'].unique()))
print(df.columns)

# 存在订单金额为0的订单，三包单或者赠送订单，不好处理
# 物料类别有冲突这里只取一种类别，后续让业务解决冲突
# 先不管这些问题，不做筛选，一起来整体看看

# 考虑去掉订单金额为0的订单，赠送订单
print('订单金额为0的疑似赠送订单数：', len(df.loc[df['订单金额']==0]))
# df = df.loc[df['订单金额']!=0]

# 考虑三包件
display(df['是否三包描述'].value_counts())
print(len(df.loc[df['是否三包描述']=='三包申请单', '物料编号'].unique()))
print(len(df.loc[df['是否三包描述']=='销售申请', '物料编号'].unique()))

20311
Index(['物料编号', '是否三包描述', '基本计量单位', '创建日期', 'year', 'month', 'week', 'day',
       '订单数量', '订单金额', '物料描述', '是否进口件', '物料类别', '周期（天）下计划后的交货周期', '在产情况',
       '吨位区间', '设备类型'],
      dtype='object')
订单金额为0的疑似赠送订单数： 52527


销售申请     256221
三包申请单     18616
Name: 是否三包描述, dtype: int64

4376
19337


In [44]:
# 根据订单金额和数量求出物料单价，以此作为成本
df['物料单价'] = df['订单金额'] / df['订单数量']

# 查看可以合并类
print(len(df['物料类别'].unique()), df['物料类别'].unique())
print(df.loc[df['物料类别'].str.contains('弹簧件'), '物料类别'].unique())

# 部分物料类别合并
# 不确定合并的对不对，另外还有诸如吊臂、支腿、两室、等不确定要不要合并
df.loc[df['物料类别']=='液压件', '物料类别'] = '液压件类'
df.loc[df['物料类别']=='大型结构件', '物料类别'] = '大型结构件类'
df.loc[df['物料类别']=='小型结构件(含非锻件五金件)', '物料类别'] = '小型结构件'
df.loc[df['物料类别']=='弹簧类', '物料类别'] = '弹簧件类'

43 ['电器件类' '吊臂(伸缩)' '油缸类' '液压件类' '未知' '吊臂(基本/顶节)' '橡塑尼龙类' '一般结构件类' '大型结构件'
 '钢丝绳类' '传动件传动操纵类' '锻件(表面处理)' '小型结构件(含非锻件五金件)' '辅材杂件类' '支腿(活动)' '轴承类'
 '小型结构件' '标准节' '管和接头类' '发动机类' '泵和阀类' '两室配件' '减速机' '底盘车桥类' '转台' '国标件'
 '标牌图册类' '液压件' '异型臂' '滤芯类' '起重设备类' '润滑与密封类' '弹簧件类' '马达类' '桅杆' '弹簧类' '油品类'
 '两室' '大型结构件类' '气动元件类' '支腿(固定)' '车架' '履带架']
['弹簧件类']


In [45]:
# 构造包含完整月份的订单数据，没有的月份订单数量补0
# 无物料类别信息等业务信息的，补未知
def struct_full_month_df(df):
    full_month_df = pd.DataFrame(product(df['物料编号'].unique(), 
                         np.sort(df['year'].unique()),
                         np.sort(df['month'].unique())),
                columns=['物料编号', 'year', 'month'])
    temp1 = df.groupby(['物料编号', 'year', 'month'], as_index=False)['订单数量'].agg({'月订单数量':'sum'})
    temp2 = df.groupby(['物料类别', '物料编号'], as_index=False).count()
    temp3 = df.groupby(['基本计量单位', '物料编号'], as_index=False).count()
    temp4 = df.groupby(['是否进口件', '物料编号'], as_index=False).count()
    temp5 = df.groupby(['周期（天）下计划后的交货周期', '物料编号'], as_index=False).count()
    temp6 = df.groupby(['在产情况', '物料编号'], as_index=False).count()
    temp7 = df.groupby(['物料描述', '物料编号'], as_index=False).count()
    temp8 = df.groupby(['吨位区间', '物料编号'], as_index=False).count()
    temp9 = df.groupby(['设备类型', '物料编号'], as_index=False).count()
    temp10 = df.groupby(['物料编号'], as_index=False)['物料单价'].agg({'物料单位成本':'max'})
    full_month_df = pd.merge(full_month_df, temp1, on=['物料编号', 'year', 'month'], how='left')
    full_month_df = pd.merge(full_month_df, temp2[['物料类别', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp3[['基本计量单位', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp4[['是否进口件', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp5[['周期（天）下计划后的交货周期', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp6[['在产情况', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp7[['物料描述', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp8[['吨位区间', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp9[['设备类型', '物料编号']], on=['物料编号'], how='left')
    full_month_df = pd.merge(full_month_df, temp10, on=['物料编号'], how='left')
    full_month_df['月订单数量'].fillna(0, inplace=True)
    full_month_df.fillna('未知', inplace=True)
    return full_month_df

def plot_monthly(data, ylabel='月订单数量', title='各年份各类配件需求总量随月份变化图'):
    
    plt.figure(figsize=(10, 3))
    x = data['date']
    y = data[ylabel]
#     plt.plot(x, y)
    plt.bar(x, y, width=5)
    plt.xticks(fontsize=10)    
    plt.yticks(fontsize=10) 
    plt.xlabel('月份', fontsize=10)    
    plt.ylabel('需求', fontsize=10)
    plt.title(title, fontsize=10)
    plt.grid()
    plt.gcf().autofmt_xdate() # 自动旋转日期标记
    plt.show()

# 思路1：计算需求间隔平均值
# 注意计算间隔的方式，算序列前面的0，不算序列后面的0，只要没到非0值就一直计数，到非0值就重新计数
# 定义是否合理，如果序列后面有很多0呢？？？？后续应该在序列最后补一个非0值来计算间隔，也有问题？？？？
# 平均间隔单独看有各种问题，需要考虑结合0值的比例，平均值等信息
# def interval_mean(input_endog):
    
#     input_series = np.asarray(input_endog)
# #     input_length = len(input_series)
#     nzd = np.where(input_series != 0)[0] # find location of non-zero demand
# #     z = input_series[nzd] # demand
#     x = np.concatenate([[nzd[0]], np.diff(nzd)]) # intervals
#     # input_series = [0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]
#     # nzd = [ 5  6  8 11 16]
#     # x = [5 1 2 3 5]
#     p = np.mean(x)
#     if list(nzd) == []: # 序列里全是0，此时，均值为0，方差为0，间隔为nan
#         p = np.nan
#     if list(nzd) == [0]: # 序列开头非0，后面全是0，此时，间隔给一个大值，20？
#         p = 20
#     # 输出求间隔平均值
#     return p

# 思路2：计算需求间隔平均值
# 考虑几个特殊情况，如果不处理，对平均间隔计算有较大影响
def interval_mean(input_endog):
    
    input_series = np.asarray(input_endog)
    # 考虑序列全0的情况，定义间隔长度为序列长度
    if np.isin(input_series, [0]).all():
        return len(input_series)
    # 考虑序列后面一个或多个0结尾的情况，补一个0来定义间隔长度
    if input_series[-1]==0:
        input_series = np.append(input_series, 1)
    nzd = np.where(input_series != 0)[0] # find location of non-zero demand
    if nzd[0]!=0:
        x = np.concatenate([[nzd[0]], np.diff(nzd)]) # intervals
    else:
        # 考虑序列开头为非0的情况，按照上面的代码会计算出间隔为0，这里针对这种情况特殊处理
        x = np.diff(nzd)
    p = np.mean(x)
    # 输出求间隔平均值
    return p

In [46]:
matrl_full = struct_full_month_df(df)
print(len(matrl_full))
matrl_full.head()

487464


,物料编号,year,month,月订单数量,物料类别,基本计量单位,是否进口件,周期（天）下计划后的交货周期,在产情况,物料描述,吨位区间,设备类型,物料单位成本
0,00630736400001010,2020,1,4.0,电器件类,PC,否,12,停产,槽型板,大吨位,汽车吊,37.81
1,00630736400001010,2020,2,4.0,电器件类,PC,否,12,停产,槽型板,大吨位,汽车吊,37.81
2,00630736400001010,2020,3,4.0,电器件类,PC,否,12,停产,槽型板,大吨位,汽车吊,37.81
3,00630736400001010,2020,4,0.0,电器件类,PC,否,12,停产,槽型板,大吨位,汽车吊,37.81
4,00630736400001010,2020,5,0.0,电器件类,PC,否,12,停产,槽型板,大吨位,汽车吊,37.81


In [39]:
# l = matrl_full.loc[matrl_full['物料编号']=='00630736400001010', '月订单数量'].tolist()
l = [0,0,0,0,0,0,1,0]
input_series = np.asarray(l)
print('原始序列', input_series)

# # 考虑序列全0的情况，定义间隔长度为序列长度
# if np.isin(input_series, [0]).all():
#     return len(input_series)
# 考虑序列后面一个或多个0结尾的情况，补一个0来定义间隔长度
# 补0操作这种合理吗，比如[0,0,0,0,0,0,1,0]和[0,0,0,0,0,0,0,1]的平均间隔就有一定差距
if input_series[-1]==0:
    input_series = np.append(input_series, 1)
nzd = np.where(input_series != 0)[0] # find location of non-zero demand
print('非0位置', nzd)
if nzd[0]!=0:
    x = np.concatenate([[nzd[0]+1], np.diff(nzd)]) # intervals
else:
    # 考虑序列开头为非0的情况，按照上面的代码会计算出间隔为0，这里针对这种情况特殊处理
    x = np.diff(nzd)
print('间隔序列', x)
p = np.mean(x)
print('间隔平均值', p)


原始序列 [0 0 0 0 0 0 1 0]
非0位置 [6 8]
间隔序列 [7 2]
间隔平均值 4.5
